In [2]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from cobra.util import create_stoichiometric_matrix
import numpy as np
import escher

In [3]:
model = read_sbml_model('iMM904.xml') 

# Intro

Querry for heme as a metabolite

In [4]:
for metabolite in model.metabolites.query('heme', 'name'):
    print(metabolite.id)

pheme_m
sheme_c


Some info on the metabolite

In [5]:
model.metabolites.pheme_m

Metabolite identifier,pheme_m
Name,Protoheme C34H30FeN4O4
Memory address,0x01f68320dda0
Formula,
Compartment,m
In 2 reaction(s),"FCLTm, HEMEOSm"


We can gather additional information (like references to external datbases) about the metabolite through the annotation attribute

In [6]:
model.metabolites.pheme_m.annotation

{'sbo': 'SBO:0000247',
 'bigg.metabolite': 'pheme',
 'biocyc': 'META:PROTOHEME',
 'chebi': ['CHEBI:60344',
  'CHEBI:26355',
  'CHEBI:17627',
  'CHEBI:5651',
  'CHEBI:14957'],
 'inchi_key': 'KABFMIBPWCXCRK-RGGAHWMASA-J',
 'kegg.compound': 'C00032',
 'metanetx.chemical': 'MNXM249',
 'reactome.compound': ['917877', '71185', '189444']}

# The heme biosynthetic pathway

We want a lot of Protoheme C34H30FeN4O4 (id: pheme_m). Let's start by figuring out what the flux to heme currently is?

We look at the reactions heme is directely involved in... ...and then work backwards to glycine (https://www.genome.jp/kegg-bin/show_pathway?sce00860)

## Last reaction (YOR176W/HEM15)

In [7]:
for reaction in model.metabolites.pheme_m.reactions:
    print(reaction.id, reaction, reaction.name) # search for the last reaction in the heme biosynthesis

FCLTm FCLTm: fe2_m + ppp9_m --> 2.0 h_m + pheme_m Ferrochelatase  mitochondrial
HEMEOSm HEMEOSm: frdp_m + h2o_m + pheme_m --> hemeO_m + ppi_m Heme O synthase  mitochondrial


In [57]:
heme_15 = model.reactions.get_by_id('FCLTm') # last reaction in the heme biosynthesis

## 2nd to last reaction (YER014W/HEM14)

In [9]:
for reaction in model.metabolites.ppp9_m.reactions:
    print(reaction.id, reaction, reaction.name) # search for the second to last reaction in the heme biosynthesis

PPPGOm PPPGOm: 3.0 o2_m + 2.0 pppg9_m --> 6.0 h2o_m + 2.0 ppp9_m Protoporphyrinogen oxidase  mitochondrial
FCLTm FCLTm: fe2_m + ppp9_m --> 2.0 h_m + pheme_m Ferrochelatase  mitochondrial


In [58]:
heme_14 = model.reactions.get_by_id('PPPGOm') # second to last reaction in the heme biosynthesis

## 3rd to last reaction (YDR044W/HEM13)

In [29]:
for reactions in model.reactions.query('Coproporphyrinogen', 'name'): # RefSeq from https://www.genome.jp/entry/sce:YDR044W 
    print(reactions.id)

CPPPGO


In [59]:
heme_13 = model.reactions.get_by_id('CPPPGO')

## 4th to last reaction (YDR047W/HEM12,HEM6)
Denne deler sig også ud til HEM14 (knockout?)

In [34]:
for reactions in model.reactions.query('Uroporphyrinogen decarboxylase', 'name'): # RefSeq from https://www.genome.jp/entry/sce:YDR044W 
    print(reactions.id)

UPPDC1


In [60]:
heme_12 = model.reactions.get_by_id('UPPDC1') 

## 5th to last reaction (YOR278W/HEM4)

In [40]:
for reactions in model.reactions.query('Uroporphyrinogen-III synthase', 'name'): # RefSeq from https://www.genome.jp/entry/sce:YOR278W
    print(reactions.id)

UPP3S


In [61]:
heme_4 = model.reactions.get_by_id('UPP3S') 

## 6th to last reaction (YDL205C/HEM3,OLE2)

In [43]:
for reactions in model.reactions.query('Hydroxymethylbilane synthase', 'name'): # RefSeq from https://www.genome.jp/entry/sce:YDL205C
    print(reactions.id)

HMBS


In [63]:
heme_3 = model.reactions.get_by_id('HMBS')

## 7th to last reaction (YGL040C/HEM2, OLE4, SLU1)

In [47]:
for reactions in model.reactions.query('Porphobilinogen synthase', 'name'): # RefSeq from https://www.genome.jp/entry/sce:YGL040C
    print(reactions.id)

PPBNGS


In [64]:
heme_2 = model.reactions.get_by_id('PPBNGS')

## 8th to last reaction (YDR232W/HEM1, CYD1, OLE3)

In [53]:
for reactions in model.reactions.query('5 aminolevulinate synthase', 'name'): # RefSeq from https://www.genome.jp/entry/sce:YDR232W
    print(reactions.id)

ALASm


In [65]:
heme_1 = model.reactions.get_by_id('ALASm')

## all reactions

Now let's make a list with the 8 reactions (shortest path) involved in the biosynthesis of heme from glycine.

In [69]:
heme_pathway = [heme_1,heme_2,heme_3,heme_4,heme_12,heme_13,heme_14,heme_15]

In [71]:
heme_pathway # the entire heme pathway

[<Reaction ALASm at 0x1f6835189e8>,
 <Reaction PPBNGS at 0x1f683f9fba8>,
 <Reaction HMBS at 0x1f683c042b0>,
 <Reaction UPP3S at 0x1f6841a4b70>,
 <Reaction UPPDC1 at 0x1f6841b0940>,
 <Reaction HMBS at 0x1f683c042b0>,
 <Reaction PPPGOm at 0x1f683f4b320>,
 <Reaction FCLTm at 0x1f683b66d30>]

## Metabolic pathway visualizations with Escher MÅSKE

In [15]:
escher.list_available_maps()
escher.Builder('iMM904.Central carbon metabolism')

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [16]:
escher.Builder('iMM904.Central carbon metabolism',
               reaction_data={'PGK': 100},
               metabolite_data={'ATP': 20})

Builder(metabolite_data={'ATP': 20}, reaction_data={'PGK': 100})

# YNB media w/o amino acids and supplemented with Fe{+3}

Most of the experimental data we are going to compare this GSMM with is based on cultivations of CEN.PK 113-7D in YNB media. Therefore, we will have to change the media.
https://www.carlroth.com/com/en/nutrient-media--molecular-biology/yeast-nitrogen-base-%28without-amino-acids%29/p/hp26.1

In [17]:
medium = model.medium
medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

# Fluxes

We want a lot of Protoheme C34H30FeN4O4 **(id: pheme_m)**. Let's start by figuring out what the flux to heme currently is.

In [33]:
solution = model.optimize()
solution.fluxes[solution.fluxes != 0.]

13GS         0.326670
2DDA7Ptm    -0.076083
ACCOAC       0.114455
AGAT_SC      0.007110
AGPRim       0.046260
              ...    
URIDK2r     -0.001727
PTRCt3i      1.506442
PTRCtex2     1.506442
PYK         17.721990
PYRDC       15.946051
Name: fluxes, Length: 304, dtype: float64

In [53]:
def get_reaction_fluxes(model):
    solution = model.optimize()
    all_fluxes = solution.fluxes[solution.fluxes != 0.]
    ls=[]
    i = 0
    while i != len(all_fluxes):
        j = (all_fluxes.index[i],all_fluxes[i])
        ls.append(j)
        i+=1
    else:
        return ls

In [55]:
get_reaction_fluxes(model) # all non-zero fluxes

[('13GS', 0.32667000056331896),
 ('2DDA7Ptm', -0.0760829054889718),
 ('ACCOAC', 0.11445540379271733),
 ('AGAT_SC', 0.007110282881489231),
 ('AGPRim', 0.0462600185852356),
 ('AGTi', 0.015000681820021441),
 ('AHCi', 0.005383088659265125),
 ('AHSERL2', 0.016494704822240202),
 ('ACGKm', 0.0462600185852356),
 ('2OBUTtm', 0.05547172110376416),
 ('2OXOADPtim', 0.0823871644000898),
 ('ACHBSm', 0.05547172110376416),
 ('ACLSm', 0.16149265977795377),
 ('AICART', 0.04758132216523195),
 ('AIRCr', 0.028495826009655598),
 ('AKGDam', 0.2210007411235192),
 ('AKGDbm', 0.2210007411235192),
 ('ALATA_L', -0.1470734666794246),
 ('ACONTm', 0.5292790018771937),
 ('ACOTAim', 0.0462600185852356),
 ('ACS', 0.14707058802238251),
 ('ALCD2ir', 15.815474819203953),
 ('3C3HMPtm', -0.08532339457787076),
 ('ADK1', 0.7150209854592826),
 ('ADK3', -0.26621532412843324),
 ('ALDD2y', 0.13057588320014232),
 ('ADNK1', 0.005383088659265125),
 ('ADSK', 0.016494704822240202),
 ('ADSL1r', 0.03336363505929052),
 ('ADSL2r', 0.02849

# Knockout of HAP1

In [31]:
for genes in model.genes.query('o', 'name'):
    print(genes.name)

In [ ]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    print('pfk knocked out: ', cobra_model.optimize())

It could be an idea to knockout the enzymes catalysing the conversion to Heme A C49H55FeN4O6 and Heme O C49H56FeN4O5

 Knockout the gene: HAP1 / systemic name: YLR256W
 OR the gene for Heme O synthase HEMEOSm

What is the flux to heme now?